From a slack message by L Grey in the `servicex-cms` chat.

In [2]:
from pathlib import Path
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from dask.distributed import Client
import dask
import dask_awkward
import awkward as ak
import hist.dask as hda

def extract_pushdown(coll):
    hlg_sorted = coll.dask._toposort_layers()
    pushdown_deps = []
    for key in hlg_sorted:
        annotations = coll.dask.layers[key].annotations
        if annotations is not None and "pushdown" in annotations:
            #print(key, coll.dask.layers[key].annotations)
            pushdown_deps = [key] + pushdown_deps
    for dep in pushdown_deps:
        layer = coll.dask.layers[dep]
        fcn = list(layer.dsk.values())[0][0]
        if isinstance(layer, dask_awkward.layers.AwkwardBlockwiseLayer) and not isinstance(layer, dask_awkward.layers.AwkwardInputLayer):
            print(layer.dsk)
            print(dir(layer))
            print(list(layer.keys()))
            print(dep, fcn.fn)
            print(dir(fcn))
            print(fcn.arg_repackers[0])
            print("")
        else:
            print(dep, fcn)

if __name__ == "__main__":
    #client = Client()


    dask.config.set({"awkward.optimization.enabled": True, "awkward.raise-failed-meta": True, "awkward.optimization.on-fail": "raise"})

    with dask.annotate(pushdown="servicex"):
        events = NanoEventsFactory.from_root(
            {
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
            },
            metadata={"dataset": "nano_dy"},
            schemaclass=NanoAODSchema,
        ).events()

        mask = events.Muon.pt > 30
        events = events[ak.any(mask, axis=1)]
        
    myhist = hda.Hist.new.Regular(50, 0, 2.5, name="abseta").Double()

    myhist.fill(abseta=abs(events.Muon.eta))

    extract_pushdown(myhist)

{'getitem-ec5f6cafeebecbdacbd91ba47ef5a331': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x000001F4A210B350>, '__dask_blockwise__0', '__dask_blockwise__1')}
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__contains__', '__copy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_cull', '_cull_dependencies', '_dict', '_repr_html_', 'annotations', 'clone', 'collection_annotations', 'concatenate', 'cull', 'dims', 'dsk', 'from_blockwise', 'get', 'get_dependencies', 'get_output_keys', 'has_been_unpickled', 'indices', 'io_deps', 'is_materialized', 'items',